In [24]:
!pip install pyradiomics

# Teste

In [25]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";

In [26]:
import numpy as np
import pandas as pd
from SimpleITK import GetImageFromArray
from radiomics import glrlm
from skimage.io import imread_collection, imread
import matplotlib.pyplot as plt
from tensorflow.image import resize_with_crop_or_pad
from glob import glob
import warnings
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [27]:
path = glob('/data/flavio/Camila/imagens/clahe/originais-clahe/*.png')
images = [imread(image) for image in path]
print('The database has {} segmented images'.format(len(images)))

The database has 10192 segmented images


In [28]:
from tqdm import tqdm
resize_i = []
for i in tqdm(images): 
           im = cv2.resize((i),(256, 256))
           resize_i.append(im)

100%|██████████| 10192/10192 [00:00<00:00, 12683.50it/s]


In [29]:
resize_i[0].shape

(256, 256)

In [30]:
#images, teste_30 = train_test_split(images1, test_size=0.30)

In [31]:
#path = glob('/data/flavio/Camila/imagens/segs/lungmask-unet/*.png')
#mask = [imread(image) for image in path]
#print('The database has {} segmented images'.format(len(mask)))

In [32]:
#masks_path = "/data/flavio/Camila/imagens/segs/lungmask-unet"


#mask_names = os.listdir(masks_path)

#testing_files = set(os.listdir(masks_path))
#def getData(X_shape, flag = "test"):
 #   mask_array = []
    
#    if flag == "test":
 #       for i in tqdm(testing_files): 
  #          mask = cv2.resize(cv2.imread(os.path.join(masks_path,i)),(258,258))[:,:,0]
            
            
   #         mask_array.append(mask)
        
    #    return mask_array
    
        

In [33]:
#masks = getData(258)

In [34]:
masksIm = []

for x in range(len(resize_i)):
    masks = np.zeros((resize_i[x].shape[0], resize_i[x].shape[1]))
    for i in range(resize_i[x].shape[0]):
        for j in range(resize_i[x].shape[1]):
            if (resize_i[x][i,j] == 0):
                masks[i,j] = 0
            else:
                masks[i,j] = 1
                
    masksIm.append(masks)

In [35]:
masksIm[0].shape

(256, 256)

In [36]:
glrlmFeat = []

for m in range(len(resize_i)):
    print(m)
    print(path[m])
    
    glrlmFeatures = glrlm.RadiomicsGLRLM(GetImageFromArray(resize_i[m]), GetImageFromArray((masksIm[m]).astype(np.uint8)), verbose=False)

    glrlmFeatures.enableFeatureByName('ShortRunEmphasis', True)
    glrlmFeatures.enableFeatureByName('LongRunEmphasis', True)
    glrlmFeatures.enableFeatureByName('GrayLevelNonUniformity', True)
    glrlmFeatures.enableFeatureByName('RunLengthNonUniformity', True)
    glrlmFeatures.enableFeatureByName('RunPercentage', True)
    glrlmFeatures.enableFeatureByName('LowGrayLevelRunEmphasis', True)
    glrlmFeatures.enableFeatureByName('HighGrayLevelRunEmphasis', True)
    glrlmFeatures.enableFeatureByName('ShortRunLowGrayLevelEmphasis', True)
    glrlmFeatures.enableFeatureByName('ShortRunHighGrayLevelEmphasis', True)
    glrlmFeatures.enableFeatureByName('LongRunLowGrayLevelEmphasis', True)
    glrlmFeatures.enableFeatureByName('LongRunHighGrayLevelEmphasis', True) 

    glrlmFeat.append(glrlmFeatures.execute())

0

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-8792.png

1

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-4052.png

2

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-8035.png

3

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-9675.png

4

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-5612.png

5

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-3059.png

6

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-6707.png

7

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-7147.png

8

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-2619.png

9

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-9214.png

10

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-5273.png

11

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-4394.png

12

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Normal-4841.png

13

/data/flavio/Camila/imagens/clahe/mask-clahe/normal/Norma

In [37]:
len(glrlmFeat)

10192

In [38]:
print('Atributos extraídos da primeira imagem')
glrlmFeat[0]

Atributos extraídos da primeira imagem


{'ShortRunEmphasis': array(0.42371599),
 'LongRunEmphasis': array(600.7320284),
 'GrayLevelNonUniformity': array(1095.07839677),
 'RunLengthNonUniformity': array(1687.24802408),
 'RunPercentage': array(0.12987518),
 'LowGrayLevelRunEmphasis': array(0.15168876),
 'HighGrayLevelRunEmphasis': array(26.63429468),
 'ShortRunLowGrayLevelEmphasis': array(0.02925679),
 'ShortRunHighGrayLevelEmphasis': array(13.20445026),
 'LongRunLowGrayLevelEmphasis': array(588.98952041),
 'LongRunHighGrayLevelEmphasis': array(901.75053792)}

In [39]:
glrlmFinal = []
orderedNames = ['ShortRunEmphasis','LongRunEmphasis','GrayLevelNonUniformity', 'RunLengthNonUniformity', 'RunPercentage', 'LowGrayLevelRunEmphasis', 
                'HighGrayLevelRunEmphasis', 'ShortRunLowGrayLevelEmphasis', 'ShortRunHighGrayLevelEmphasis', 'LongRunLowGrayLevelEmphasis', 
                'LongRunHighGrayLevelEmphasis']

for features in glrlmFeat:
    for names in orderedNames:
        glrlmFinal.append(features.get(names).item())

glrlmFinal = np.reshape(glrlmFinal, (10192, 11))

In [40]:
glrlmFinal[0][0]

0.42371599186158987

In [41]:
nome_file = '/data/flavio/Camila/csv_clahe/glrlm/normal/mask/glrlm_normal_comseg'
matriz_csv = pd.DataFrame(glrlmFinal)

In [42]:
matriz_csv.to_csv(nome_file+'.csv', index=False)

In [43]:
matriz_csv 

,0,1,2,3,4,5,6,7,8,9,10
0,0.423716,600.732028,1095.078397,1687.248024,0.129875,0.151689,26.634295,0.029257,13.204450,588.989520,901.750538
1,0.377137,1400.964044,717.679111,845.793560,0.078445,0.149987,42.755536,0.010814,18.021641,1387.880432,2076.328871
2,0.382524,1247.174544,728.735216,904.365063,0.082497,0.153231,37.434256,0.013627,15.741695,1234.573047,1832.669393
3,0.517294,1463.762321,606.854549,1307.895040,0.076561,0.201654,27.404796,0.036590,16.549024,1458.622963,1628.693846
4,0.413507,828.363736,965.976674,1366.241615,0.109951,0.156295,26.164539,0.025314,12.774471,816.341974,1126.887442
...,...,...,...,...,...,...,...,...,...,...,...
10187,0.469443,972.965838,869.331938,1505.163682,0.102127,0.166181,27.840899,0.032529,14.948425,964.399804,1240.602722
10188,0.408259,943.258692,880.240312,1235.039526,0.100403,0.160215,27.813040,0.024123,13.472525,932.041097,1259.225590
10189,0.451641,798.369246,1026.807979,1549.844881,0.112312,0.144000,28.549666,0.022382,15.350070,787.882699,1081.820528
10190,0.402613,798.599207,919.145976,1317.973558,0.110317,0.131424,39.527582,0.014492,17.806221,785.164991,1381.444221


In [44]:
#np.save('./glrlm', glrlmFinal)